In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [58]:
banka = pd.read_csv(r"Cases\Bankruptcy\Bankruptcy.csv",
                   index_col = 0)
banka.head()

,D,YR,R1,R2,R3,R4,R5,R6,R7,R8,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
NO,,,,,,,,,,,,,,,,,,,,,
1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,10.36,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,3.13,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,2.41,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,5.55,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,2.85,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06


In [7]:
X = banka.drop(['D', 'YR'] , axis =1)
y = banka['D']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify = y,
                                                    random_state = 2022,
                                                    train_size = 0.7)

In [9]:
mm = MinMaxScaler()

In [35]:
mlp = MLPClassifier(activation = "logistic",
                   random_state = 2022)

In [40]:
pipe = Pipeline([("MM", mm), ("MLP", mlp)])
pipe.fit(X_train, y_train)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('MM', MinMaxScaler()),
                ('MLP',
                 MLPClassifier(activation='logistic', random_state=2022))])

In [41]:
y_pred = pipe.predict(X_test)
y_pred_prob = pipe.predict_proba(X_test)[:,1]

In [42]:
print(accuracy_score(y_test, y_pred))

0.825


In [43]:
print(roc_auc_score(y_test, y_pred_prob))

0.9


# Grid Search CV

In [44]:
pipe = Pipeline([("MM", mm), ("MLP", mlp)])
print(pipe.get_params())

{'memory': None, 'steps': [('MM', MinMaxScaler()), ('MLP', MLPClassifier(activation='logistic', random_state=2022))], 'verbose': False, 'MM': MinMaxScaler(), 'MLP': MLPClassifier(activation='logistic', random_state=2022), 'MM__clip': False, 'MM__copy': True, 'MM__feature_range': (0, 1), 'MLP__activation': 'logistic', 'MLP__alpha': 0.0001, 'MLP__batch_size': 'auto', 'MLP__beta_1': 0.9, 'MLP__beta_2': 0.999, 'MLP__early_stopping': False, 'MLP__epsilon': 1e-08, 'MLP__hidden_layer_sizes': (100,), 'MLP__learning_rate': 'constant', 'MLP__learning_rate_init': 0.001, 'MLP__max_fun': 15000, 'MLP__max_iter': 200, 'MLP__momentum': 0.9, 'MLP__n_iter_no_change': 10, 'MLP__nesterovs_momentum': True, 'MLP__power_t': 0.5, 'MLP__random_state': 2022, 'MLP__shuffle': True, 'MLP__solver': 'adam', 'MLP__tol': 0.0001, 'MLP__validation_fraction': 0.1, 'MLP__verbose': False, 'MLP__warm_start': False}


In [52]:
params = { "MLP__hidden_layer_sizes": [(20, 10, 5), (10, 5), (50,)],
         "MLP__activation": ["tanh", "logistic", "identity"],
         "MLP__learning_rate": ["constant", "invscaling", "adaptive"],
         "MLP__learning_rate_init": [0.001, 0.3, 0.5]}

In [53]:
kfold = StratifiedKFold(n_splits = 5,
                       shuffle = True,
                       random_state =2022)

In [54]:
gcv = GridSearchCV(pipe,
                  param_grid = params,
                  cv = kfold,
                  scoring = "roc_auc",
                  verbose = 3)

gcv.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.863 total time=   0.1s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.940 total time=   0.1s
[CV 3/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.633 total time=   0.1s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.876 total time=   0.1s
[CV 5/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.001;, score=0.917 total time=   0.1s
[CV 1/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.3;, score=0.280 total time=   0.0s
[CV 2/5] END MLP__activation=tanh, MLP__hidden_layer_sizes=(20, 10, 5), MLP__learning_rate=constant, MLP__learning_rate_init=0.3;, score=0.797 total time=   0.0s


...

[CV 3/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.639 total time=   0.0s
[CV 4/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.970 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 5/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.001;, score=0.893 total time=   0.0s
[CV 1/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.857 total time=   0.0s
[CV 2/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.901 total time=   0.0s
[CV 3/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.757 total time=   0.0s
[CV 4/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.852 total time=   0.0s
[CV 5/5] END MLP__activation=identity, MLP__hidden_layer_sizes=(50,), MLP__learning_rate=adaptive, MLP__learning_rate_init=0.3;, score=0.935 total time=   0.0s
[CV 1/5] END MLP__activation=identity,

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=Pipeline(steps=[('MM', MinMaxScaler()),
                                       ('MLP',
                                        MLPClassifier(activation='logistic',
                                                      random_state=2022))]),
             param_grid={'MLP__activation': ['tanh', 'logistic', 'identity'],
                         'MLP__hidden_layer_sizes': [(20, 10, 5), (10, 5),
                                                     (50,)],
                         'MLP__learning_rate': ['constant', 'invscaling',
                                                'adaptive'],
                         'MLP__learning_rate_init': [0.001, 0.3, 0.5]},
             scoring='roc_auc', verbose=3)

In [55]:
print(gcv.best_params_)

{'MLP__activation': 'tanh', 'MLP__hidden_layer_sizes': (10, 5), 'MLP__learning_rate': 'constant', 'MLP__learning_rate_init': 0.001}


In [57]:
print(gcv.best_score_)

0.8679628064243449
